In [1]:
using BenchmarkTools, Test

using Revise, LazySets
using LazySets.Arrays

┌ Info: Recompiling stale cache file /home/mforets/.julia/compiled/v1.2/LazySets/NjrGc.ji for LazySets [b4f0291d-fe17-52bc-9479-3d1a343d9043]
└ @ Base loading.jl:1240


## Removing output type annotations (#1840)

In [2]:
x = [rand(Interval) for _ in 1:4]

4-element Array{Interval{Float64,IntervalArithmetic.Interval{Float64}},1}:
 Interval{Float64,IntervalArithmetic.Interval{Float64}}([0.101965, 0.192478])  
 Interval{Float64,IntervalArithmetic.Interval{Float64}}([0.283072, 0.37474])   
 Interval{Float64,IntervalArithmetic.Interval{Float64}}([-0.171448, -0.116653])
 Interval{Float64,IntervalArithmetic.Interval{Float64}}([-1.13171, -0.138705]) 

In [3]:
a, b = x[1], x[2]

(Interval{Float64,IntervalArithmetic.Interval{Float64}}([0.101965, 0.192478]), Interval{Float64,IntervalArithmetic.Interval{Float64}}([0.283072, 0.37474]))

In [8]:
@btime intersection($a, $b)

  0.016 ns (0 allocations: 0 bytes)


EmptySet{Float64}()

In [14]:
function intersection_no_output_annotation(x::Interval{N}, y::Interval{N}) where {N<:Real}
    if min(y) > max(x) || min(x) > max(y)
        return EmptySet{N}()
    else
        return Interval(max(min(x), min(y)), min(max(x), max(y)))
    end
end

function intersection_output_annotation(x::Interval{N}, y::Interval{N})::Union{Interval{N}, EmptySet{N}} where {N<:Real}
    if min(y) > max(x) || min(x) > max(y)
        return EmptySet{N}()
    else
        return Interval(max(min(x), min(y)), min(max(x), max(y)))
    end
end

intersection_output_annotation (generic function with 1 method)

In [15]:
@code_warntype intersection_no_output_annotation(a, b)

Variables
  #self#::Core.Compiler.Const(intersection_no_output_annotation, false)
  x::Interval{Float64,IntervalArithmetic.Interval{Float64}}
  y::Interval{Float64,IntervalArithmetic.Interval{Float64}}
  @_4::Bool

Body::Union{EmptySet{Float64}, Interval{Float64,IntervalArithmetic.Interval{Float64}}}
1 ─ %1  = Main.min(y)::Float64
│   %2  = Main.max(x)::Float64
│   %3  = (%1 > %2)::Bool
└──       goto #3 if not %3
2 ─       (@_4 = %3)
└──       goto #4
3 ─ %7  = Main.min(x)::Float64
│   %8  = Main.max(y)::Float64
└──       (@_4 = %7 > %8)
4 ┄       goto #6 if not @_4
5 ─ %11 = Core.apply_type(Main.EmptySet, $(Expr(:static_parameter, 1)))::Core.Compiler.Const(EmptySet{Float64}, false)
│   %12 = (%11)()::Core.Compiler.Const(EmptySet{Float64}(), false)
└──       return %12
6 ─ %14 = Main.min(x)::Float64
│   %15 = Main.min(y)::Float64
│   %16 = Main.max(%14, %15)::Float64
│   %17 = Main.max(x)::Float64
│   %18 = Main.max(y)::Float64
│   %19 = Main.min(%17, %18)::Float64
│   %20 = Main.Inte

In [16]:
@code_warntype intersection_output_annotation(a, b)

Variables
  #self#::Core.Compiler.Const(intersection_output_annotation, false)
  x::Interval{Float64,IntervalArithmetic.Interval{Float64}}
  y::Interval{Float64,IntervalArithmetic.Interval{Float64}}
  @_4::Bool

Body::Union{EmptySet{Float64}, Interval{Float64,IntervalArithmetic.Interval{Float64}}}
1 ─ %1  = Core.apply_type(Main.Interval, $(Expr(:static_parameter, 1)))::Core.Compiler.Const(Interval{Float64,IN} where IN<:IntervalArithmetic.AbstractInterval{Float64}, false)
│   %2  = Core.apply_type(Main.EmptySet, $(Expr(:static_parameter, 1)))::Core.Compiler.Const(EmptySet{Float64}, false)
│   %3  = Core.apply_type(Main.Union, %1, %2)::Core.Compiler.Const(Union{EmptySet{Float64}, Interval{Float64,IN} where IN<:IntervalArithmetic.AbstractInterval{Float64}}, false)
│   %4  = Main.min(y)::Float64
│   %5  = Main.max(x)::Float64
│   %6  = (%4 > %5)::Bool
└──       goto #3 if not %6
2 ─       (@_4 = %6)
└──       goto #4
3 ─ %10 = Main.min(x)::Float64
│   %11 = Main.max(y)::Float64
└──       (

## Revision of Ball1 (#1894)

Loop invariant https://github.com/JuliaReach/LazySets.jl/pull/1894

In [4]:
using LazySets: _leq, _geq

In [5]:
function _in_failfast(x::AbstractVector{N}, B::Ball1{N}, failfast::Bool=false) where {N<:Real}
    @assert length(x) == dim(B) "a $(length(x))-dimensional vector is " *
        "incompatible with a $(dim(B))-dimensional set"
    sum = zero(N)
    for (i, xi) in enumerate(x)
        @inbounds sum += abs(B.center[i] - xi)
        if failfast && !_leq(sum, B.radius)
            return false
        end
    end
    return _leq(sum, B.radius)
end

_in_failfast (generic function with 2 methods)

In [6]:
function _in_failfast_v2(x::AbstractVector{N}, B::Ball1{N}, failfast::Bool=false) where {N<:Real}
    @assert length(x) == dim(B) "a $(length(x))-dimensional vector is " *
        "incompatible with a $(dim(B))-dimensional set"
    sum = zero(N)
    if failfast
        @inbounds for (i, xi) in enumerate(x)
            sum += abs(B.center[i] - xi)
            !_leq(sum, B.radius) && return false
        end
    else
        @inbounds for (i, xi) in enumerate(x)
            sum += abs(B.center[i] - xi)
        end
    end
    return _leq(sum, B.radius)
end

_in_failfast_v2 (generic function with 2 methods)

In [7]:
function _in_failfast_v3(x::AbstractVector{N}, B::Ball1{N}) where {N<:Real}
    @assert length(x) == dim(B) "a $(length(x))-dimensional vector is " *
        "incompatible with a $(dim(B))-dimensional set"
    sum = mapreduce(abs, +, B.center .- x)
    return _leq(sum, B.radius)
end

_in_failfast_v3 (generic function with 1 method)

In [8]:
function _in_failfast_v4(x::AbstractVector{N}, B::Ball1{N}) where {N<:Real}
    @assert length(x) == dim(B) "a $(length(x))-dimensional vector is " *
        "incompatible with a $(dim(B))-dimensional set"
    sum = zero(N)
    @inbounds @simd for i in eachindex(x)
        sum += abs(B.center[i] - x[i])
    end
    return _leq(sum, B.radius)
end

_in_failfast_v4 (generic function with 1 method)

In [26]:
#=
function _in_failfast_v5(x::AbstractVector{N}, B::Ball1{N}, failfast::Bool=false) where {N<:Real}
    @assert length(x) == dim(B) "a $(length(x))-dimensional vector is " *
        "incompatible with a $(dim(B))-dimensional set"
    sum = zero(N)
    if failfast        
        @inbounds @simd for i in eachindex(x)
            sum += abs(B.center[i] - x[i])
            !_leq(sum, B.radius) && return false
        end
    elseif failfast
        @inbounds @simd for i in eachindex(x)
            sum += abs(B.center[i] - x[i])
        end
    end
    return _leq(sum, B.radius)
end

@inline function _in_failfast_v5_false(x::AbstractVector{N}, B::Ball1{N}, failfast::Bool=false) where {N<:Real}
    @inbounds @simd for i in eachindex(x)
        sum += abs(B.center[i] - x[i])
    end
    return sum
end

@inline function _in_failfast_v5_true(x::AbstractVector{N}, B::Ball1{N}, failfast::Bool=false) where {N<:Real}

end
=#

_in_failfast_v5_true (generic function with 1 method)

In [2]:
using LoopVectorization

┌ Info: Precompiling LoopVectorization [bdcacae8-1622-11e9-2a5c-532679323890]
└ @ Base loading.jl:1242


In [18]:
function _in_failfast_v6(x::AbstractVector{N}, B::Ball1{N}) where {N<:Real}
    @assert length(x) == dim(B) "a $(length(x))-dimensional vector is " *
        "incompatible with a $(dim(B))-dimensional set"
    sum = zero(N)
    c = B.center
    @avx for i in eachindex(x)
        sum += abs(c[i] - x[i])
    end
    return _leq(sum, B.radius)
end

_in_failfast_v6 (generic function with 1 method)

In [21]:
n = 10

B = Ball1(zeros(n), 1.0)
x_true = rand(n)*0.001
x_false_best = vcat(2., rand(n-1)*0.001)
x_false_worse = vcat(rand(n-1)*0.001, 2.)

for x in [x_true, x_false_best, x_false_worse]
    println("---")
    @btime in($x, $B)
    @btime _in_failfast($x, $B, $true)
    @btime _in_failfast($x, $B, $false)
    @btime _in_failfast_v2($x, $B, $true)
    @btime _in_failfast_v2($x, $B, $false)
    @btime _in_failfast_v3($x, $B)
    @btime _in_failfast_v4($x, $B)
    #@btime _in_failfast_v5($x, $B, $true)
    #@btime _in_failfast_v5($x, $B, $false)
    @btime _in_failfast_v6($x, $B)
end

---
  9.299 ns (0 allocations: 0 bytes)
  14.429 ns (0 allocations: 0 bytes)
#= In[21]:11 =# @btime(_in_failfast($(Expr(:$, :x)), $(Expr(:$, :B)), $(Expr(:$, true)))) = true
  9.338 ns (0 allocations: 0 bytes)
  12.964 ns (0 allocations: 0 bytes)
  8.813 ns (0 allocations: 0 bytes)
  34.471 ns (1 allocation: 160 bytes)
  9.221 ns (0 allocations: 0 bytes)
  9.610 ns (0 allocations: 0 bytes)
---
  9.306 ns (0 allocations: 0 bytes)
  9.226 ns (0 allocations: 0 bytes)
#= In[21]:11 =# @btime(_in_failfast($(Expr(:$, :x)), $(Expr(:$, :B)), $(Expr(:$, true)))) = false
  15.793 ns (0 allocations: 0 bytes)
  8.788 ns (0 allocations: 0 bytes)
  14.068 ns (0 allocations: 0 bytes)
  37.685 ns (1 allocation: 160 bytes)
  15.034 ns (0 allocations: 0 bytes)
  15.561 ns (0 allocations: 0 bytes)
---
  9.307 ns (0 allocations: 0 bytes)
  17.523 ns (0 allocations: 0 bytes)
#= In[21]:11 =# @btime(_in_failfast($(Expr(:$, :x)), $(Expr(:$, :B)), $(Expr(:$, true)))) = false
  15.014 ns (0 allocations: 0 bytes)

## https://github.com/JuliaReach/LazySets.jl/pull/1880

In [8]:
N = Float64

In [4]:
h_flat = Hyperrectangle(N[1, 2, 3, 4, 5], N[1, 0, 2, 0, 3])

Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}}([1.0, 2.0, 3.0, 4.0, 5.0], [1.0, 0.0, 2.0, 0.0, 3.0])

In [9]:
@test collect(generators(h_flat)) ==
    [SingleEntryVector(1, 5, N(1)), SingleEntryVector(3, 5, N(2)),
     SingleEntryVector(5, 5, N(3))]

Test Passed

In [10]:
@test genmat(h_flat) == N[1 0 0; 0 0 0; 0 2 0; 0 0 0; 0 0 3]

Test Failed at In[10]:1
  Expression: genmat(h_flat) == $(Expr(:typed_vcat, :N, :(1 0 0), :(0 0 0), :(0 2 0), :(0 0 0), :(0 0 3)))
   Evaluated: 
  [1, 1]  =  1.0
  [3, 3]  =  2.0
  [5, 5]  =  3.0 == [1.0 0.0 0.0; 0.0 0.0 0.0; … ; 0.0 0.0 0.0; 0.0 0.0 3.0]


Test.FallbackTestSetException: There was an error during testing

In [11]:
@test ngens(h_flat) == 3

Test Passed

In [12]:
genmat(h_flat)

5×5 SparseArrays.SparseMatrixCSC{Float64,Int64} with 3 stored entries:
  [1, 1]  =  1.0
  [3, 3]  =  2.0
  [5, 5]  =  3.0

## Faster constraints list of interval

In [2]:
x = rand(Interval)

Interval{Float64,IntervalArithmetic.Interval{Float64}}([0.34905, 0.837542])

In [3]:
import IntervalArithmetic

N = Float64

# random interval
rand(Interval)

# constructor from IntervalArithmetic.Interval
x = Interval(IntervalArithmetic.Interval(N(0), N(1)))

# constructor from a vector
x = Interval(N[0, 1])

# type-less constructor
x = Interval(N(0), N(1))

@test dim(x) == 1
@test center(x) == N[0.5]
@test min(x) == N(0) && max(x) == N(1)
v = vertices_list(x)
@test N[0] in v && N[1] in v
# test interface method an_element and membership
@test an_element(x) ∈ x
# test containment
@test x ⊆ x

#x ⊆ N(0.2)*x
#@which constraints_list(N(0.2)*x)
#linear_map(hcat(N(0.2)...), x)
#@test x ⊈ N(0.2) * x

#=

@test x ⊆ N(2) * x
@test x ⊆ Interval(N(0), N(2))
@test x ⊈ Interval(N(-1), N(0.5))
=#

Test Passed

In [4]:
@which constraints_list(x)

constraints_list(H::AbstractHyperrectangle{N}) where N<:Real in LazySets at /home/mforets/.julia/dev/LazySets/src/Interfaces/AbstractHyperrectangle.jl:243

In [65]:
function constraints_list2(x::Interval{N}) where {N<:Real}
    constraints = Vector{LinearConstraint{N, SingleEntryVector{N}}}(undef, 2)
    e₁ = SingleEntryVector(1, 1, one(N))
    constraints[1] = HalfSpace(e₁, max(x))
    constraints[2] = HalfSpace(-e₁, -min(x))
    return constraints
end

function constraints_list2(lm::LinearMap{N, IN}) where {N<:Real, IN<:Interval{N}}
    constraints = Vector{LinearConstraint{N, SingleEntryVector{N}}}(undef, 2)
    y = linear_map(lm.M[1, 1] * lm.X.dat) # existe?
    e₁ = SingleEntryVector(1, 1, one(N))
    constraints[1] = HalfSpace(e₁, max(y))
    constraints[2] = HalfSpace(-e₁, -min(y))
    return constraints
end

constraints_list2 (generic function with 3 methods)

In [60]:
N = Float64

M = hcat(N(0.2)...)
x = rand(Interval)

Interval{Float64,IntervalArithmetic.Interval{Float64}}([-0.218559, 0.404922])

In [82]:
@btime linear_map($M, $x)

  291.309 ns (12 allocations: 976 bytes)


Zonotope{Float64}([0.011293052659236686], [0.09570771886516959])

In [63]:
lm = M * x
@btime constraints_list($x)

  34.973 ns (1 allocation: 144 bytes)


2-element Array{HalfSpace{Float64,SingleEntryVector{Float64}},1}:
 HalfSpace{Float64,SingleEntryVector{Float64}}([1.0], 0.4049218496653462)  
 HalfSpace{Float64,SingleEntryVector{Float64}}([-1.0], 0.21855808387635195)

In [ ]:
lm = M * x
@btime constraints_list2($lm)

In [81]:
vertices_list(linear_map(M, x))

2-element Array{Array{Float64,1},1}:
 [0.19488184317425938]
 [0.18018079474341364]

In [79]:
@btime linear_map2($M, $x)

  12.956 ns (0 allocations: 0 bytes)


Interval{Float64,IntervalArithmetic.Interval{Float64}}([0.18018, 0.194882])

In [76]:
function linear_map2(M::AbstractMatrix{N}, x::Interval{N}) where {N<:Real}
    @assert size(M) == (1, 1) "a linear map of size $(size(M)) " *
        "cannot be applied to an interval"

    return Interval(M[1] * x.dat)
end

function linear_map2(α::Real, x::Interval{N}) where {N<:Real}
    return Interval(α * x.dat)
end

linear_map2 (generic function with 2 methods)

In [77]:
constraints_list2(N(0.2) * x)

2-element Array{HalfSpace{Float64,SingleEntryVector{Float64}},1}:
 HalfSpace{Float64,SingleEntryVector{Float64}}([1.0], 0.1948818431742594)  
 HalfSpace{Float64,SingleEntryVector{Float64}}([-1.0], -0.1801807947434136)

In [43]:
linear_map()

1-element SingleEntryVector{Float64}:
 1.0

In [81]:
@btime constraints_list($lm)

  46.839 ns (1 allocation: 144 bytes)


2-element Array{HalfSpace{Float64,SingleEntryVector{Float64}},1}:
 HalfSpace{Float64,SingleEntryVector{Float64}}([1.0], 0.08098436993306926)
 HalfSpace{Float64,SingleEntryVector{Float64}}([-1.0], 0.0437116167752704)

In [77]:
@btime constraints_list($lm)

  47.073 ns (1 allocation: 144 bytes)


2-element Array{HalfSpace{Float64,SingleEntryVector{Float64}},1}:
 HalfSpace{Float64,SingleEntryVector{Float64}}([1.0], 0.08098436993306926)
 HalfSpace{Float64,SingleEntryVector{Float64}}([-1.0], 0.0437116167752704)

In [40]:
x = LazySets.rand(Interval)

Interval{Float64,IntervalArithmetic.Interval{Float64}}([0.900903, 0.97441])

In [41]:
@btime constraints_list($x)

  99.178 ns (4 allocations: 432 bytes)


2-element Array{HalfSpace{Float64,SingleEntryVector{Float64}},1}:
 HalfSpace{Float64,SingleEntryVector{Float64}}([1.0], 0.9744092158712969)  
 HalfSpace{Float64,SingleEntryVector{Float64}}([-1.0], -0.9009039737170681)

In [46]:
@btime constraints_list2($x)

  35.006 ns (1 allocation: 144 bytes)


2-element Array{HalfSpace{Float64,SingleEntryVector{Float64}},1}:
 HalfSpace{Float64,SingleEntryVector{Float64}}([1.0], 0.9744092158712969)  
 HalfSpace{Float64,SingleEntryVector{Float64}}([-1.0], -0.9009039737170681)

## Some interval specializations (#1902, #1903, #1904, #1905)

In [1]:
using BenchmarkTools
using Revise, LazySets

┌ Info: Recompiling stale cache file /home/mforets/.julia/compiled/v1.2/LazySets/NjrGc.ji for LazySets [b4f0291d-fe17-52bc-9479-3d1a343d9043]
└ @ Base loading.jl:1240


In [2]:
x = rand(Interval)

Interval{Float64,IntervalArithmetic.Interval{Float64}}([0.191275, 0.39985])

In [3]:
M = hcat(0.2...)

1×1 Array{Float64,2}:
 0.2

In [5]:
lm = M * x
@btime constraints_list($lm)

  49.486 ns (1 allocation: 144 bytes)


2-element Array{HalfSpace{Float64,LazySets.Arrays.SingleEntryVector{Float64}},1}:
 HalfSpace{Float64,LazySets.Arrays.SingleEntryVector{Float64}}([1.0], 0.07996995699017853)   
 HalfSpace{Float64,LazySets.Arrays.SingleEntryVector{Float64}}([-1.0], -0.038255126602414216)

In [4]:
@btime linear_map($M, $x)

  13.203 ns (0 allocations: 0 bytes)


Interval{Float64,IntervalArithmetic.Interval{Float64}}([0.0382551, 0.07997])

In [17]:
M = rand(1, 1)

1×1 Array{Float64,2}:
 0.8369266396792854

In [20]:
@btime LazySets._linear_map_interval($M, $x)

  9.291 ns (0 allocations: 0 bytes)


Interval{Float64,IntervalArithmetic.Interval{Float64}}([0.160083, 0.334645])

In [21]:
@btime LazySets.linear_map($M, $x)

  13.368 ns (0 allocations: 0 bytes)


Interval{Float64,IntervalArithmetic.Interval{Float64}}([0.160083, 0.334645])

In [13]:
@code_warntype LazySets._linear_map_interval(M, x)

Variables
  #self#::Core.Compiler.Const(LazySets._linear_map_interval, false)
  M::Array{Float64,2}
  x::Interval{Float64,IntervalArithmetic.Interval{Float64}}
  α::Float64

Body::Interval{Float64,IntervalArithmetic.Interval{Float64}}
1 ─      (α = Base.getindex(M, 1, 1))
│   %2 = α::Float64
│   %3 = Base.getproperty(x, :dat)::IntervalArithmetic.Interval{Float64}
│   %4 = (%2 * %3)::IntervalArithmetic.Interval{Float64}
│   %5 = LazySets.Interval(%4)::Interval{Float64,IntervalArithmetic.Interval{Float64}}
└──      return %5


In [18]:
function linear_map2(M::AbstractMatrix{N}, x::Interval{N}) where {N<:Real}
    @assert size(M) == (1, 1) "a linear map of size $(size(M)) " *
        "cannot be applied to an interval"
    α = M[1, 1]
    return Interval(α * x.dat)
end

linear_map2 (generic function with 1 method)

In [ ]:
@btime linear_map($M, $x)

In [22]:
@which linear_map(M, x)

linear_map(M::AbstractArray{N,2}, Z::AbstractZonotope{N}) where N<:Real in LazySets at /home/mforets/.julia/dev/LazySets/src/Interfaces/AbstractZonotope.jl:286

In [19]:
@btime linear_map2($M, $x)

  13.365 ns (0 allocations: 0 bytes)


Interval{Float64,IntervalArithmetic.Interval{Float64}}([0.0997541, 0.200817])

In [20]:
(13 - 10) / 13.

0.23076923076923078

In [42]:
genmat(x)

1×1 Array{Float64,2}:
 0.25265552355874943

In [184]:
import IntervalArithmetic

function linear_map3(M::AbstractMatrix{N}, x::Interval{N}) where {N<:Real}
    @assert size(M, 2) == 1 "a linear map of size $(size(M)) " *
        "cannot be applied to an interval"
    nout = size(M, 1)
    if nout == 1
        α = M[1, 1]
        return Interval(α * x.dat)
    else
        cx = IntervalArithmetic.mid(x.dat)
        gx = cx - min(x)
        c = Vector{N}(undef, nout)
        gen = Matrix{N}(undef, nout, 1)
        @inbounds for i in 1:nout
            c[i] = M[i, 1] * cx
            gen[i] = M[i, 1] * gx
        end
        return Zonotope(c, gen, remove_zero_generators=false)
    end
end

function linear_map4(M::AbstractMatrix{N}, x::Interval{N}) where {N<:Real}
    @assert size(M, 2) == 1 "a linear map of size $(size(M)) " *
        "cannot be applied to an interval"
    nout = size(M, 1)
    if nout == 1
        α = M[1, 1]
        return Interval(α * x.dat)
    else
        cx = IntervalArithmetic.mid(x.dat)
        gx = cx - min(x)
        c = M[:, 1]
        gen = copy(M)
        scal!(nout, cx, c, 1)
        scal!(nout, gx, gen, nout)
        return Zonotope(c, gen, remove_zero_generators=false)
    end
end

linear_map4 (generic function with 1 method)

In [182]:
M = rand(3, 1)
strides(M)

(1, 3)

In [ ]:
scal!(n, a, X, incx)

In [178]:
using LinearAlgebra.BLAS

In [180]:
scal!(, cx, )

search: scal! scale scal UniformScaling @specialize @nospecialize



```
scal!(n, a, X, incx)
```

Overwrite `X` with `a*X` for the first `n` elements of array `X` with stride `incx`. Returns `X`.


In [93]:
(high(x) - low(x))/2

1-element Array{Float64,1}:
 0.4510716455013524

In [51]:
M = rand(2, 2)

2×2 Array{Float64,2}:
 0.307817  0.023415
 0.941367  0.352579

In [54]:
M .* 2.0

2×2 Array{Float64,2}:
 0.615634  0.0468301
 1.88273   0.705159 

In [148]:
A = fill(1, (5, 1));
strides(A)

(1, 5)

In [58]:
α = 0.2
@btime linear_map2($α, $x)

  0.016 ns (0 allocations: 0 bytes)


Interval{Float64,IntervalArithmetic.Interval{Float64}}([-0.0743651, 0.0150274])

In [174]:
M = rand(3, 1)
@btime linear_map3($M, $x)

  84.611 ns (3 allocations: 256 bytes)


Zonotope{Float64}([-0.058094962263273306, -0.15356772174908628, -0.05060785596912751], [0.16791726216600072; 0.44387104128439536; 0.1462765838441631])

In [ ]:
M = rand(3, 1)
@btime linear_map4($M, $x)

In [64]:
xarray = [rand(Interval) for _ in 1:1000]
Marray = [rand(1, 1) for _ in 1:1000];

In [60]:
giybeltype(xarray)

mmmmmm

Interval{Float64,IntervalArithmetic.Interval{Float64}}

In [136]:
function ftest2(xarray, Marray)
    m = length(xarray)
    out = Vector{eltype(xarray)}(undef, m)
    for i in 1:m
        out[i] = linear_map2(Marray[i], xarray[i])
    end
    return out
end

function ftest3(xarray, Marray)
    m = length(xarray)
    out = Vector{eltype(xarray)}(undef, m)
    for i in 1:m
        out[i] = linear_map3(Marray[i], xarray[i])
    end
    return out
end

ftest3 (generic function with 1 method)

In [137]:
@btime ftest2($xarray, $Marray);

  14.762 μs (1 allocation: 15.75 KiB)


In [138]:
@btime ftest3($xarray, $Marray);

  15.742 μs (1 allocation: 15.75 KiB)


In [133]:
M = rand(3, 1)

3×1 Array{Float64,2}:
 0.19694858201512666
 0.643213909622034  
 0.9632750616502088 

In [135]:
@code_warntype linear_map2(M, x)

Variables
  #self#::Core.Compiler.Const(linear_map2, false)
  M::Array{Float64,2}
  x::Interval{Float64,IntervalArithmetic.Interval{Float64}}
  α::Float64

Body::Interval{Float64,IntervalArithmetic.Interval{Float64}}
1 ─       Core.NewvarNode(:(α))
│   %2  = Main.size(M)::Tuple{Int64,Int64}
│   %3  = Core.tuple(1, 1)::Core.Compiler.Const((1, 1), false)
│   %4  = (%2 == %3)::Bool
└──       goto #3 if not %4
2 ─       goto #4
3 ─ %7  = Base.getproperty(Base.Main, :Base)::Any
│   %8  = Base.getproperty(%7, :string)::Any
│   %9  = Main.size(M)::Tuple{Int64,Int64}
│   %10 = Base.string("a linear map of size ", %9, " ")::String
│   %11 = (%10 * "cannot be applied to an interval")::String
│   %12 = (%8)(%11)::Any
│   %13 = Base.AssertionError(%12)::AssertionError
└──       Base.throw(%13)
4 ┄       (α = Base.getindex(M, 1, 1))
│   %16 = α::Float64
│   %17 = Base.getproperty(x, :dat)::IntervalArithmetic.Interval{Float64}
│   %18 = (%16 * %17)::IntervalArithmetic.Interval{Float64}
│   %19 = Mai

In [88]:
x = rand(Interval)

@code_warntype genmat(x)

Variables
  #self#::Core.Compiler.Const(LazySets.genmat, false)
  H::Interval{Float64,IntervalArithmetic.Interval{Float64}}

Body::Any
1 ─ %1 = LazySets.genmat_fallback(H)::Any
└──      return %1


In [45]:
@btime linear_map($M, $x)

  633.599 ns (14 allocations: 1.03 KiB)


Zonotope{Float64}([0.11652764080795253, 0.3671650234020733, 0.3080553572195031], [0.039180617310919374; 0.12345356150804025; 0.1035788502891599])

## Constraints list of the lazy linear map of an interval

In [6]:
using BenchmarkTools
using Revise, LazySets

In [2]:
M = rand(1, 1)
x = rand(Interval)

Interval{Float64,IntervalArithmetic.Interval{Float64}}([-0.121519, 0.631573])

In [3]:
linear_map(M, x)

Interval{Float64,IntervalArithmetic.Interval{Float64}}([-0.0307416, 0.159775])

In [4]:
lm = LinearMap(M, x)

LinearMap{Float64,Interval{Float64,IntervalArithmetic.Interval{Float64}},Float64,Array{Float64,2}}([0.2529784452615631], Interval{Float64,IntervalArithmetic.Interval{Float64}}([-0.121519, 0.631573]))

In [5]:
constraints_list(lm)

2-element Array{HalfSpace{Float64,LazySets.Arrays.SingleEntryVector{Float64}},1}:
 HalfSpace{Float64,LazySets.Arrays.SingleEntryVector{Float64}}([1.0], 0.159774284299269)   
 HalfSpace{Float64,LazySets.Arrays.SingleEntryVector{Float64}}([-1.0], 0.03074151731426427)

In [7]:
@btime constraints_list($lm)

  48.488 ns (1 allocation: 144 bytes)


2-element Array{HalfSpace{Float64,LazySets.Arrays.SingleEntryVector{Float64}},1}:
 HalfSpace{Float64,LazySets.Arrays.SingleEntryVector{Float64}}([1.0], 0.159774284299269)   
 HalfSpace{Float64,LazySets.Arrays.SingleEntryVector{Float64}}([-1.0], 0.03074151731426427)

In [9]:
@btime constraints_list($x)

  38.289 ns (1 allocation: 144 bytes)


2-element Array{HalfSpace{Float64,LazySets.Arrays.SingleEntryVector{Float64}},1}:
 HalfSpace{Float64,LazySets.Arrays.SingleEntryVector{Float64}}([1.0], 0.6315727181186241) 
 HalfSpace{Float64,LazySets.Arrays.SingleEntryVector{Float64}}([-1.0], 0.1215183265217697)

In [13]:
using Polyhedra

┌ Info: Recompiling stale cache file /home/mforets/.julia/compiled/v1.2/Polyhedra/17i4E.ji for Polyhedra [67491407-f73d-577b-9b50-8179a7c68029]
└ @ Base loading.jl:1240


6-element Array{HalfSpace{Float64,VN} where VN<:AbstractArray{Float64,1},1}:
 HalfSpace{Float64,Array{Float64,1}}([0.0, -0.3415190547182963, 0.6584809452817035], -0.0)                                  
 HalfSpace{Float64,Array{Float64,1}}([-0.3265149953696854, 0.3380385937858283, 0.1753226449987087], -3.963039859995711e-17) 
 HalfSpace{Float64,Array{Float64,1}}([-0.0, 0.3415190547182963, -0.6584809452817035], 0.0)                                  
 HalfSpace{Float64,Array{Float64,1}}([0.3265149953696854, -0.3380385937858283, -0.1753226449987087], 3.963039859995711e-17) 
 HalfSpace{Float64,Array{Float64,1}}([0.3690135343185548, 0.28087907646907045, 0.14567704255257874], 0.3596824428903503)    
 HalfSpace{Float64,Array{Float64,1}}([-0.3664454675176457, -0.27892436163000006, -0.14466323589823807], 0.06872341256483284)

In [14]:
constraints_list(rand(3, 1) * x)

6-element Array{HalfSpace{Float64,VN} where VN<:AbstractArray{Float64,1},1}:
 HalfSpace{Float64,Array{Float64,1}}([0.0, -0.02520191276369132, 0.9747980872363086], -0.0)                                   
 HalfSpace{Float64,Array{Float64,1}}([-0.5301412888689088, 0.3106369069006847, 0.008031041844869925], 0.0)                    
 HalfSpace{Float64,Array{Float64,1}}([-0.0, 0.02520191276369132, -0.9747980872363086], 0.0)                                   
 HalfSpace{Float64,Array{Float64,1}}([0.5301412888689088, -0.3106369069006847, -0.008031041844869925], -0.0)                  
 HalfSpace{Float64,Array{Float64,1}}([0.21939999196824533, 0.3741838140975719, 0.009673949881465308], 0.14200786050244352)    
 HalfSpace{Float64,Array{Float64,1}}([-0.20816908264671352, -0.3550296452755285, -0.009178748159144802], 0.025924502078614315)

## Unbounded polyhedra plots

https://github.com/JuliaReach/LazySets.jl/issues/576

In [1]:
using Polyhedra, Plots

# MathematicalSystems

## Rename some map fields  (#128)

## System macro

In [ ]:
using Revise, MathematicalSystems, LazySets

In [24]:
macro f(expr)
    println(expr.args[1])
    println(expr.args[2])
end

@f (macro with 2 methods)

In [25]:
@f x' = A*x + c, x ∈ X

x'
(A * x + c, x ∈ X)
